<a href="https://colab.research.google.com/github/EnoshNyarige/spam_model_deployment/blob/master/Spam_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# ! ls "/content/gdrive/My Drive"

In [4]:
df = pd.read_csv("/content/gdrive/My Drive/spam_or_not_spam_dataset/spam_or_not_spam.csv")

In [5]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [6]:
import re
import nltk
from keras.layers import Flatten, Dense
from keras.layers import Embedding,LSTM
from keras.models import Sequential
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
df['label'].value_counts()

0    2500
1     500
Name: label, dtype: int64

we have 500 hams and 2500 spams

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

In [10]:
X = df['email'].fillna(' ')
y = df['label']

In [11]:
# X = df['email'].values
# y = df['label'].values

In [12]:
def preprocess(text):
    text = text.str.lower()
    text = text.str.replace(r'\r', ' ')
    text = text.str.replace(r'\n', ' ')
    text = text.str.replace('[^a-zA-Z0-9]', ' ')
    return text

In [13]:
x = preprocess(X)
x = x.values
y = y.values

In [14]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state=0, shuffle = True, stratify= df['label'])
# xtrain

In [15]:
# xtest

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenizer_sequences(X):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = len(tokenizer.word_index) + 1
    
    return tokenizer, sequences

In [17]:
tokenizer, sequences = tokenizer_sequences(xtrain)

In [18]:
# integer encode the documents
train_set_encoded = tokenizer.texts_to_sequences(xtrain)
test_set_encoded = tokenizer.texts_to_sequences(xtest)
print(train_set_encoded[0:2])

[[4617, 39, 21, 2032, 633, 3, 45, 11, 8, 232, 6, 667, 181, 94, 36, 4617, 718, 200, 304, 23, 410, 6, 851, 1170, 379, 13, 257, 20, 78, 4617, 1, 4617, 634, 39, 1238, 870, 46, 81, 1, 3, 76, 12, 2, 181, 2, 4617, 1171, 317, 3, 19, 3, 285, 2, 95, 45, 12, 179, 121, 302, 4617, 3, 20, 6, 466, 4303, 3, 103, 614, 12, 49, 6, 536, 41, 39, 4617, 109, 285, 13, 14, 6, 200, 18, 1073, 5, 799, 1, 1381, 4617, 2033, 1330, 58, 11943, 12, 819, 1331, 81, 70, 53, 8, 97, 7, 40, 568, 600, 34, 11, 371, 5, 436, 3, 8284, 51, 1412, 11944, 8, 102, 506, 39, 437, 2, 1469, 6509, 6510, 213, 43, 5, 133, 8285, 16016, 11, 39, 8286, 1470, 14, 5893, 339, 60, 101, 4, 492, 16, 168, 2604, 70, 13, 102, 2076, 51, 183, 10, 39, 98, 1870, 39, 71, 23, 121, 3, 45, 15, 4, 506, 81, 2, 5894, 800, 58, 18, 36, 16017, 495, 5, 5895, 2, 1526, 16018, 39, 4618, 1870, 1720, 60, 15, 22, 204, 388, 9, 275, 204, 388, 9, 16, 801, 5, 788, 9, 41, 8, 48, 6, 181, 22, 8287, 2209, 190, 8288, 25, 39, 98, 425, 69, 1871, 51, 39, 583, 3, 2, 1332, 5, 3472, 51, 38

In [19]:
import pickle
# saving
with open('vec_tokenizer.sav', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
max_length = 6
padded_train_set = pad_sequences(train_set_encoded, maxlen=max_length, padding='post')
padded_test_set = pad_sequences(test_set_encoded, maxlen=max_length, padding='post')
# print(padded_test_set)

In [25]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-12-07 11:25:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-07 11:25:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-07 11:25:37--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [26]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [27]:
!ls

gdrive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip	vec_tokenizer.sav


In [30]:
# load the whole embedding into memory
embeddingz = {}
f = open('/content/glove.6B.50d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddingz[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddingz))

Loaded 400000 word vectors.


In [32]:
#weight matrix of words in training docs
embedding_matrix = np.zeros((sequences, 50))
len(embedding_matrix)
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddingz.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector 

In [33]:
from keras.optimizers import Adam
opt = Adam(lr = 0.001)
loss = 'binary_crossentropy'

In [35]:
model = Sequential()
model.add(Embedding(sequences, 50, weights=[embedding_matrix], input_length=6))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 50)             1516100   
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                3010      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 1,519,121
Trainable params: 1,519,121
Non-trainable params: 0
_________________________________________________________________


In [36]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=100, verbose=1)

Epoch 1/100
75/75 [==============================] - 1s 13ms/step - loss: 0.5729 - accuracy: 0.7237
Epoch 2/100
75/75 [==============================] - 1s 13ms/step - loss: 0.3356 - accuracy: 0.8696
Epoch 3/100
75/75 [==============================] - 1s 13ms/step - loss: 0.2609 - accuracy: 0.9017
Epoch 4/100
75/75 [==============================] - 1s 12ms/step - loss: 0.2058 - accuracy: 0.9271
Epoch 5/100
75/75 [==============================] - 1s 13ms/step - loss: 0.1634 - accuracy: 0.9446
Epoch 6/100
75/75 [==============================] - 1s 13ms/step - loss: 0.1309 - accuracy: 0.9592
Epoch 7/100
75/75 [==============================] - 1s 13ms/step - loss: 0.1058 - accuracy: 0.9638
Epoch 8/100
75/75 [==============================] - 1s 13ms/step - loss: 0.0881 - accuracy: 0.9692
Epoch 9/100
75/75 [==============================] - 1s 13ms/step - loss: 0.0729 - accuracy: 0.9742
Epoch 10/100
75/75 [==============================] - 1s 13ms/step - loss: 0.0653 - accuracy: 0.9746

In [37]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.2437427043914795 test accuracy:  92.66666769981384 %


In [38]:
from tensorflow.keras.layers import Dropout

In [40]:
opt = Adam(lr = 0.0001)
loss = 'binary_crossentropy'

# Lets optimize our model
model = Sequential()
model.add(Embedding(sequences, 50, weights=[embedding_matrix], input_length=6))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(10, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid' ))
# compile network
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])
# summarize defined model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 6, 50)             1516100   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,577,521
Trainable params: 1,577,521
Non-trainable params: 0
____________________________________________

In [41]:
#Fitting the data into the model
model.fit(padded_train_set, ytrain, epochs=100, verbose=1)

Epoch 1/100
75/75 [==============================] - 1s 14ms/step - loss: 0.5471 - accuracy: 0.7725
Epoch 2/100
75/75 [==============================] - 1s 15ms/step - loss: 0.4296 - accuracy: 0.8329
Epoch 3/100
75/75 [==============================] - 1s 15ms/step - loss: 0.4066 - accuracy: 0.8325
Epoch 4/100
75/75 [==============================] - 1s 15ms/step - loss: 0.3814 - accuracy: 0.8342
Epoch 5/100
75/75 [==============================] - 1s 15ms/step - loss: 0.3605 - accuracy: 0.8404
Epoch 6/100
75/75 [==============================] - 1s 15ms/step - loss: 0.3305 - accuracy: 0.8583
Epoch 7/100
75/75 [==============================] - 1s 15ms/step - loss: 0.3022 - accuracy: 0.8833
Epoch 8/100
75/75 [==============================] - 1s 15ms/step - loss: 0.2830 - accuracy: 0.8942
Epoch 9/100
75/75 [==============================] - 1s 15ms/step - loss: 0.2610 - accuracy: 0.9033
Epoch 10/100
75/75 [==============================] - 1s 15ms/step - loss: 0.2457 - accuracy: 0.9121

In [43]:
# Lets evaluate our model
loss, accuracy = model.evaluate(padded_test_set, ytest, verbose=0)
print("validation loss: ", loss, "test accuracy: ", accuracy*100,"%")

validation loss:  0.2630668580532074 test accuracy:  94.33333277702332 %


In [44]:
# save model
model.save('spam_model.h5')

In [45]:
# load model
from keras.models import load_model
model = load_model( 'spam_model.h5' )

In [46]:
# downloadin the model and use it in Heroku
from google.colab import files
files.download('spam_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>